In [10]:
from astropy.time import Time, TimeDelta
from poliastro.twobody import Orbit
from poliastro.bodies import Earth
from astropy import units as u
import numpy as np
import matplotlib.pyplot  as plt
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(suppress=True, precision=6)

In [13]:
def cholupdate(L, W, beta):
    """ updating sqrt_cov directly """
    r = np.shape(W)[1]
    m = np.shape(L)[0]
    for i in range(r):
        L_out = np.copy(L)
        b = 1.0
        for j in range(m):
            Ljj_pow2 = L[j, j]**2
            wji_pow2 = W[j, i]**2
            L_out[j, j] = np.sqrt(Ljj_pow2 + (beta / b) * wji_pow2)
            upsilon = (Ljj_pow2 * b) + (beta * wji_pow2)
            for k in range(j+1, m):
                W[k, i] -= (W[j, i] / L[j,j]) * L[k,j]
                L_out[k, j] = ((L_out[j, j] / L[j, j]) * L[k,j]) + (L_out[j, j] * beta * W[j, i] * W[k, i] / upsilon)
            b += beta * (wji_pow2 / Ljj_pow2)
        L = np.copy(L_out)
    return L_out

def forwardsubs(A, B):
    """"""
    # x_ik = (b_ik - Sum_aij_xjk) / a_ii
    N = np.shape(A)[0]
    X = np.zeros((B.shape[0], B.shape[1]))
    for k in range(B.shape[1]):
        for i in range(N):
            sum_aij_xj = B[i, k]
            for j in range(i):
                sum_aij_xj -= A[i, j] * X[j, k]                
            X[i, k] = sum_aij_xj / A[i, i]
    return X

def backsubs(A, B):
    # x_ik = (b_ik - Sum_aij_xjk) / a_ii    
    N = np.shape(A)[0]
    X = np.zeros((B.shape[0], B.shape[1]))    
    for k in range(B.shape[1]):
        for i in range(N-1, -1, -1):
            sum_aij_xj = B[i, k]
            for j in range(N-1, i, -1):
                sum_aij_xj -= A[i, j] * X[j, k]
            X[i, k] = sum_aij_xj / A[i, i]
    return X

class SquareRootUKF(object):
    def __init__(self, x, P, Q, R):              
        self.dim_x = np.shape(x)[0]
        self.n_sigmas = (2 * self.dim_x) + 1
        
        self.kappa = 3 - self.dim_x
        
        self.sigma_scale = np.sqrt(self.dim_x + self.kappa)
        
        self.W0 = self.kappa / (self.dim_x + self.kappa)
        self.Wm = 0.5 / (self.dim_x + self.kappa)
        
        self.x = x
        
        # lower triangular matrices
        print(f"init: p shape:{P.shape}")
        
        self.sqrt_P = np.linalg.cholesky(P) # sqrt of state cov mat
        print(f"sqrt_P shape {self.sqrt_P.shape}")
        self.sqrt_Q = np.linalg.cholesky(Q) # sqrt of process noise
        self.sqrt_R = np.linalg.cholesky(R) # sqrt of meas noise
        print(f'R = \n{R}\n')
    
    def predict(self, fx_prop, dt):
        # generate sigma points
        sigmas_X = self.sigma_points(self.x, self.sqrt_P)
        print("first sigma points")
        print(sigmas_X)
        
        # propagate sigma points through the nonlinear function
        for i in range(self.n_sigmas):
            sigmas_X[:, i] = fx_prop(sigmas_X[:, i], dt)
        
        # calculate weighted mean
        x_pred = self.W0 * sigmas_X[:, 0]
        for i in range(1, self.n_sigmas):
            x_pred += self.Wm * sigmas_X[:, i]

        
        
        # build compound matrix for square-root covariance update
        # sigmas_X[:, 0] is not added because W0 could be zero which will lead to undefined outcome from sqrt(W0).
        C = (sigmas_X[:, 1:].T - x_pred) * np.sqrt(self.Wm)
        C = np.concatenate((C, self.sqrt_Q.T), axis=0)
        
        # calculate square-root covariance S using QR decomposition of compound matrix C
        # including the process noise covariance
        Q , S_minus = np.linalg.qr(C)
        print(f'Q = \n{Q}\n')
        print(f'R = \n{S_minus}\n')
        
        # Rank-1 cholesky update
        x_dev = sigmas_X[:, 0] - x_pred  # get the deviation only from the sigma point
        x_dev = np.reshape(x_dev, [-1, 1])
        print(f'x_dev = \n{x_dev}\n')
        S_minus = cholupdate(S_minus.T, x_dev, self.W0)
        
        # overwrite member x and S
        self.x = x_pred
        self.sqrt_P = S_minus
        
        print(f'S^- = \n{S_minus}\n')
    
    
    def correct(self, hx, z):
        # generate sigma points X
        sigmas_X = self.sigma_points(self.x, self.sqrt_P)
        
        # propagate sigma points X through the nonlinear function
        # to get output sigma points Y
        dim_z = np.shape(z)[0]
        sigmas_Y = np.zeros((dim_z, self.n_sigmas))
        for i in range(self.n_sigmas):
            sigmas_Y[:, i] = hx(sigmas_X[:, i])
        
        print(f'Ys = \n{sigmas_Y}\n')
        
        # calculate weighted mean y
        y_bar = self.W0 * sigmas_Y[:, 0]
        for i in range(1, self.n_sigmas):
            y_bar += self.Wm * sigmas_Y[:, i]
    
        print(f'y = \n{y_bar}\n')
        
        # build compound matrix for square-root covariance update      
        C = (sigmas_Y[:, 1:].T - y_bar) * np.sqrt(self.Wm)       
        C = np.concatenate((C, self.sqrt_R.T), axis=0)
        
        print(f'sqrt_R.T = \n{self.sqrt_R.T}\n')
        
        # calculate square-root covariance S using QR decomposition of compound matrix C
        # including the process noise covariance
        _ , S_y = np.linalg.qr(C)
        
        # Rank-1 cholesky update
        y_dev = sigmas_Y[:, 0] - y_bar
        y_dev = np.reshape(y_dev, [-1, 1])
        S_y = cholupdate(S_y.T, y_dev, self.W0)
        print(f'Sy = \n{S_y}\n')
           
        # calculate cross-correlation
        Pxy = self.calculate_cross_correlation(self.x, sigmas_X, y_bar, sigmas_Y)
        print(f'Pxy = \n{Pxy}\n')
        
        # Kalman gain calculation with two nested least-squares
        # Step1: Forward-substitution -> K = Sy \ Pxy  (since S_y is lower-triangular)
        # Step2: Backward-substitution -> K = Sy.T \ K (since S_y.T is upper-triangular)
        K = forwardsubs(S_y, Pxy)
        K = backsubs(S_y.T, K)
        print(f'K = \n{K}\n')
        
        # update state vector x
        self.x += K @ (z - y_bar)
        
        # update state square-root covariance Sk
        # S_y must be upper triangular matrix at this place
        U = K @ S_y
        
        #self.sqrt_P = r_rank_cholupdate_v2(self.sqrt_P, U, -1.0)
        self.sqrt_P = cholupdate(self.sqrt_P, U, -1.0)
    
    
    def sigma_points(self, x, sqrt_P):
        
        '''
        generating sigma points matrix x_sigma given mean 'x' and square-root covariance 'S'
        '''
        print(self.dim_x)
        sigmas_X = np.zeros((self.dim_x, self.n_sigmas))       
        print(sigmas_X.shape)
        sigmas_X[:, 0] = x

        print(sqrt_P.shape)
        print(x.shape)

        for i in range(self.dim_x):
            idx_1 = i + 1
            idx_2 = i + self.dim_x + 1
            
            sigmas_X[:, idx_1] = x + (self.sigma_scale * sqrt_P[:, i])
            sigmas_X[:, idx_2] = x - (self.sigma_scale * sqrt_P[:, i])
            
        return sigmas_X
    
    
    def calculate_cross_correlation(self, x, x_sigmas, y, y_sigmas):
        xdim = np.shape(x)[0]
        ydim = np.shape(y)[0]
        
        n_sigmas = np.shape(x_sigmas)[1]
    
        dx = (x_sigmas[:, 0] - x).reshape([-1, 1])
        dy = (y_sigmas[:, 0] - y).reshape([-1, 1])
        Pxy = self.W0 * (dx @ dy.T)
        for i in range(1, n_sigmas):
            dx = (x_sigmas[:, i] - x).reshape([-1, 1])
            dy = (y_sigmas[:, i] - y).reshape([-1, 1])
            Pxy += self.Wm * (dx @ dy.T)
    
        return Pxy

In [14]:
def linear_func(x, dt):
    F = np.eye(2)
    F[0, 1]  = dt
    return F @ x

dt = 0.1
x = np.array([0., 0.])
P = np.array([[2.0, -0.8], [-0.8, 2.0]])

Q = np.array([[3, 0], [0, 3]])
R = np.array([[1, 0], [0, 1]])

sr_ukf = SquareRootUKF(x, P, Q, R)

sr_ukf.predict(linear_func, dt)
print(f'P+Q = \n{P+Q}')

init: p shape:(2, 2)
sqrt_P shape (2, 2)
R = 
[[1 0]
 [0 1]]

2
(2, 5)
(2, 2)
(2,)
first sigma points
[[ 0.        2.44949   0.       -2.44949   0.      ]
 [ 0.       -0.979796  2.244994  0.979796 -2.244994]]
Q = 
[[-0.435465  0.126825]
 [-0.041574 -0.418046]
 [ 0.435465 -0.126825]
 [ 0.041574  0.418046]
 [-0.785674 -0.096346]
 [-0.       -0.780399]]

R = 
[[-2.204541  0.272166]
 [ 0.       -2.219443]]

x_dev = 
[[0.]
 [0.]]

S^- = 
[[ 2.204541  0.      ]
 [-0.272166  2.219443]]

P+Q = 
[[ 5.  -0.8]
 [-0.8  5. ]]


In [15]:
npzfile = np.load('GPS_meas.npz', allow_pickle=True)
noisy_measurement = npzfile['measurements']
t_measurement = npzfile['t_measurements']
# Convert measurement times to Time objects
t_measurement_time = Time(t_measurement)

N = len(t_measurement) # Number of measurements.

# Calculate time intervals (dt) between consecutive measurements
dt_list = (t_measurement_time[1:] - t_measurement_time[:-1]) # TimeDelta object. Shape: (720,)

In [16]:
# Define the state transition function
def fx(x, dt):
    orbit = Orbit.from_vectors(Earth, x[:3] * u.km, x[3:] * u.km / u.s)
    orbit = orbit.propagate(TimeDelta(dt * u.s)) # seconds!
    new_pos, new_vel = orbit.r.to(u.km).value, orbit.v.to(u.km / u.s).value
    return np.hstack((new_pos, new_vel))

# Define the measurement function
def hx(x):
    return x

In [17]:
# Initial state vector
position = [3235.64171524, 2693.72565982, -5335.42793567] 
velocity = [-4.87430005, 5.89879341, 0.01977648] 
x = np.hstack((position, velocity)) # initial state vector. Shape: (6,)

# Process & measurement noise
P = np.eye(6)
Q = np.eye(6)*10**-6 # Process noise covariance. Shape: (6, 6)
Q[:3,:3] *= 10**3
R = np.eye(6)*10**0 # Measurement noise covariance. Shape: (6, 6)
R[:3,:3] *= 10**3

sr_ukf = SquareRootUKF(x, P, Q, R)

for i in range (1, N):
    print(f"Iteration {i}")
    sr_ukf.predict(fx, dt_list[i].sec)

init: p shape:(6, 6)
sqrt_P shape (6, 6)
R = 
[[1000.    0.    0.    0.    0.    0.]
 [   0. 1000.    0.    0.    0.    0.]
 [   0.    0. 1000.    0.    0.    0.]
 [   0.    0.    0.    1.    0.    0.]
 [   0.    0.    0.    0.    1.    0.]
 [   0.    0.    0.    0.    0.    1.]]

Iteration 1
6
(6, 13)
(6, 6)
(6,)
first sigma points
[[ 3235.641715  3237.373766  3235.641715  3235.641715  3235.641715  3235.641715  3235.641715  3233.909664  3235.641715  3235.641715  3235.641715  3235.641715  3235.641715]
 [ 2693.72566   2693.72566   2695.457711  2693.72566   2693.72566   2693.72566   2693.72566   2693.72566   2691.993609  2693.72566   2693.72566   2693.72566   2693.72566 ]
 [-5335.427936 -5335.427936 -5335.427936 -5333.695885 -5335.427936 -5335.427936 -5335.427936 -5335.427936 -5335.427936 -5337.159986 -5335.427936 -5335.427936 -5335.427936]
 [   -4.8743      -4.8743      -4.8743      -4.8743      -3.142249    -4.8743      -4.8743      -4.8743      -4.8743      -4.8743      -6.606351    -

C:\Users\hylee\AppData\Local\Temp\ipykernel_23292\1906752194.py:11: RuntimeWarning: invalid value encountered in sqrt
  L_out[j, j] = np.sqrt(Ljj_pow2 + (beta / b) * wji_pow2)


AssertionError: 